In [76]:
import numpy as np

first need some decorator to prime the coroutine and it should also register the component's config data, name, etc when it is put into a chain. 

In [11]:
def cta_component(func, **kwargs):
    pass

one way this could be done in a non-object oriented way, just using 2 functions (the init is optional if it's something small) and a coroutine to wrap them:

In [15]:
def init_algo( param1=None, param2=None ):
    setupdata = dict(x=3,y=4)
    return setupdata

def process_algo( setupdata, data, param1=None, param2=None):
    val = 1
    return val

@cta_component
def algo(identifier, target, **kwargs):
    setupdata = init_algo( **kwargs )    
    while True:
        chunk = (yield)
        process_algo(setupdata, chunk, **kwargs)
        target.send(chunk)

```python
if __name__ == 'main':
    chain = algo( algo2( writer("out.fits"), param1=1 ) )
    fits_driver( "in.fits" , chain )
```

Loading camera info

In [23]:
from astropy.table import Table

camtable = Table.read("chercam.fits.gz", hdu="CHERCAM").group_by("CAM_ID")
camgeom = {}
for cam in camtable.groups:
    camgeom[cam['CAM_ID'][0]] = cam

In [25]:
print "Cameras loaded: ", camgeom.keys()


Cameras loaded:  [1, 2, 3, 4]


In [26]:
camgeom[2]

CAM_ID,PIX_ID,PIX_POSX,PIX_POSY,PIX_AREA,PIX_DIAM,PIX_NEIG [6],DUMMY
,,m,m,m2,m,,
int16,int64,float64,float64,float64,float64,int16,int16
2,0,-0.304499983788,0.636528491974,0.00149149994832,0.0414999984205,4 .. -1,0
2,1,-0.262499988079,0.636528491974,0.00149149994832,0.0414999984205,0 .. -1,0
2,2,-0.220499992371,0.636528491974,0.00149149994832,0.0414999984205,6 .. -1,0
2,3,-0.178499981761,0.636528491974,0.00149149994832,0.0414999984205,7 .. -1,0
2,4,-0.325499981642,0.600155472755,0.00149149994832,0.0414999984205,8 .. -1,0
2,5,-0.283499985933,0.600155472755,0.00149149994832,0.0414999984205,4 .. 1,0
2,6,-0.241499990225,0.600155472755,0.00149149994832,0.0414999984205,5 .. 2,0
2,7,-0.199499979615,0.600155472755,0.00149149994832,0.0414999984205,6 .. 20,0


In [ ]:
import matplotlib.pyplot as plt
import mpld3
from matplotlib import patches

In [77]:
%matplotlib inline
plt.figure(figsize=(5,5))
plt.scatter(camgeom[2]['PIX_POSX'], camgeom[2]['PIX_POSY'], marker='o',alpha=0.4)
# show pixel 100's neighbors
neighbormask = camgeom[2]['PIX_NEIG'][100]
neighbormask = neighbormask[neighbormask>=0] # get rid of -1 entries
plt.scatter(camgeom[2]['PIX_POSX'][neighbormask], camgeom[2]['PIX_POSY'][neighbormask], 
            marker='o', color='red')
mpld3.display()

In [83]:
# A nicer camera plot with filled hexagon pixels
cam = camgeom[2]
rad = cam['PIX_DIAM']/2.0
plt.figure(figsize=(5,5))
ax = plt.subplot(111)
ax.set_xlim(cam['PIX_POSX'].min()-0.1, cam['PIX_POSX'].max()+0.1)
ax.set_ylim(cam['PIX_POSY'].min()-0.1, cam['PIX_POSY'].max()+0.1)
ax.set_xlabel( "X ({})".format( cam['PIX_POSX'].unit ))
ax.set_ylabel( "Y ({})".format( cam['PIX_POSY'].unit ))

pixels = np.array([patches.RegularPolygon( (X['PIX_POSX'],X['PIX_POSY']),6,radius=X['PIX_DIAM']/2.0) 
              for X in cam])
for pix in pixels:
    pix.set_facecolor("blue")
    pix.set_edgecolor(None)
    ax.add_patch(pix)
    
for pix in pixels[neighbormask]:
    pix.set_facecolor("red")
    
mpld3.display()